In [4]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
class Args:
    seed = 333

In [6]:
data = pd.read_csv('../../Dataset/Home/dataTrain.csv')
df_test = TabularDataset("../../Dataset/Home/dataA.csv")

In [7]:
data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f38,f39,f40,f41,f42,f43,f44,f45,f46,label
0,81167,0,1,mid,0,0,0,153,0,0,...,0,0,0,0,0,0,0,624,1539,0
1,50408,1,1,mid,0,0,21,0,0,0,...,0,0,0,0,0,0,0,186,366,0
2,9114,0,0,high,36,36,120,0,0,0,...,0,0,0,0,0,0,0,24,48,1
3,53228,1,1,low,0,0,0,0,0,0,...,0,0,0,0,0,0,3,3,9,0
4,56280,1,1,mid,9,51,294,0,0,0,...,0,0,0,0,0,0,0,42,141,0


In [8]:
cat_columns = ['f3']
for col in cat_columns:
    lb = LabelEncoder()
    lb.fit(data[col])
    data[col] = lb.transform(data[col])
    df_test[col] = lb.transform(df_test[col])

In [9]:
data = data[:50000]

In [10]:
data['f47'] = data['f1'] * 10 + data['f2']

# 暴力Feature位置
loc_f = ['f1', 'f2', 'f4', 'f5', 'f6']
for i in range(len(loc_f)):
    for j in range(i + 1, len(loc_f)):
        data[f'{loc_f[i]}+{loc_f[j]}'] = data[loc_f[i]] + data[loc_f[j]]
        data[f'{loc_f[i]}-{loc_f[j]}'] = data[loc_f[i]] - data[loc_f[j]]
        data[f'{loc_f[i]}*{loc_f[j]}'] = data[loc_f[i]] * data[loc_f[j]]
        data[f'{loc_f[i]}/{loc_f[j]}'] = data[loc_f[i]] / (data[loc_f[j]]+1)

# 暴力Feature通话
com_f = ['f43', 'f44', 'f45', 'f46']
for i in range(len(com_f)):
    for j in range(i + 1, len(com_f)):
        data[f'{com_f[i]}+{com_f[j]}'] = data[com_f[i]] + data[com_f[j]]
        data[f'{com_f[i]}-{com_f[j]}'] = data[com_f[i]] - data[com_f[j]]
        data[f'{com_f[i]}*{com_f[j]}'] = data[com_f[i]] * data[com_f[j]]
        data[f'{com_f[i]}/{com_f[j]}'] = data[com_f[i]] / (data[com_f[j]]+1)

In [11]:
data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f44*f45,f44/f45,f44+f46,f44-f46,f44*f46,f44/f46,f45+f46,f45-f46,f45*f46,f45/f46
0,81167,0,1,2,0,0,0,153,0,0,...,0,0.00,1539,-1539,0,0.0,2163,-915,960336,0.405195
1,50408,1,1,2,0,0,21,0,0,0,...,0,0.00,366,-366,0,0.0,552,-180,68076,0.506812
2,9114,0,0,0,36,36,120,0,0,0,...,0,0.00,48,-48,0,0.0,72,-24,1152,0.489796
3,53228,1,1,1,0,0,0,0,0,0,...,9,0.75,12,-6,27,0.3,12,-6,27,0.300000
4,56280,1,1,2,9,51,294,0,0,0,...,0,0.00,141,-141,0,0.0,183,-99,5922,0.295775


In [12]:
train_data = data.drop(columns=["id"])
# best seed:999
train, test = train_test_split(train_data, test_size=0.2, random_state=Args.seed)

In [13]:
# 训练集中表示标签列的名称
label = "label"
# 模型存储位置
path = "../../Result/Home/AG/models/"
# 指标
metric = 'roc_auc'
# 这里不需要drop掉标签列，因为autogluon会自动识别标签列
predictor = TabularPredictor(label=label, path=path, eval_metric=metric).fit(train)
predictor.leaderboard(test, silent=True)

Beginning AutoGluon training ...
AutoGluon will save models to "../../Result/Home/AG/models/\"
AutoGluon Version:  0.5.2
Python Version:     3.8.9
Operating System:   Windows
Train Data Rows:    40000
Train Data Columns: 111
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    23175.08 MB
	Train Data (Original)  Memory Usage: 35.36 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtype

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.913629,0.908039,0.020005,0.008001,1.346302,0.020005,0.008001,1.346302,1,True,3
1,WeightedEnsemble_L2,0.913214,0.909767,0.741164,0.213047,68.099537,0.003000,0.001001,1.017229,2,True,14
2,CatBoost,0.912932,0.908759,0.006001,0.004000,4.795916,0.006001,0.004000,4.795916,1,True,7
3,LightGBM,0.912650,0.906948,0.016003,0.007001,0.886199,0.016003,0.007001,0.886199,1,True,4
4,LightGBMLarge,0.911536,0.905126,0.023005,0.009002,1.925432,0.023005,0.009002,1.925432,1,True,13
5,XGBoost,0.911101,0.906428,0.042009,0.012003,1.173264,0.042009,0.012003,1.173264,1,True,11
6,NeuralNetTorch,0.909665,0.906630,0.221050,0.069016,35.213958,0.221050,0.069016,35.213958,1,True,12
7,NeuralNetFastAI,0.908122,0.906951,0.198045,0.054011,23.544643,0.198045,0.054011,23.544643,1,True,10
8,ExtraTreesGini,0.904329,0.894222,0.237241,0.067012,1.050239,0.237241,0.067012,1.050239,1,True,8
9,ExtraTreesEntr,0.903781,0.895835,0.251054,0.065015,1.008226,0.251054,0.065015,1.008226,1,True,9


In [14]:
# 读取模型，默认使用最好模型
predictor=TabularPredictor.load("../../Result/Home/AG/models/")
# 处理测试数据
df_test['f47'] = df_test['f1'] * 10 + df_test['f2']

# 暴力Feature位置
loc_f = ['f1', 'f2', 'f4', 'f5', 'f6']
for i in range(len(loc_f)):
    for j in range(i + 1, len(loc_f)):
        df_test[f'{loc_f[i]}+{loc_f[j]}'] = df_test[loc_f[i]] + df_test[loc_f[j]]
        df_test[f'{loc_f[i]}-{loc_f[j]}'] = df_test[loc_f[i]] - df_test[loc_f[j]]
        df_test[f'{loc_f[i]}*{loc_f[j]}'] = df_test[loc_f[i]] * df_test[loc_f[j]]
        df_test[f'{loc_f[i]}/{loc_f[j]}'] = df_test[loc_f[i]] / (df_test[loc_f[j]]+1)
        
# 暴力Feature通话
com_f = ['f43', 'f44', 'f45', 'f46']
for i in range(len(com_f)):
    for j in range(i + 1, len(com_f)):
        df_test[f'{com_f[i]}+{com_f[j]}'] = df_test[com_f[i]] + df_test[com_f[j]]
        df_test[f'{com_f[i]}-{com_f[j]}'] = df_test[com_f[i]] - df_test[com_f[j]]
        df_test[f'{com_f[i]}*{com_f[j]}'] = df_test[com_f[i]] * df_test[com_f[j]]
        df_test[f'{com_f[i]}/{com_f[j]}'] = df_test[com_f[i]] / (df_test[com_f[j]]+1)    

df_test_id = df_test["id"]
df_test_feature = df_test.drop(columns=["id"])
# 预测
pred = predictor.predict_proba(df_test_feature)[1]
# 保存结果
result = pd.DataFrame({
    'id': df_test_id,
    "label": pred})
# 保存成csv文件
result.to_csv(f"../../Result/Home/AG/submission/submission_{Args.seed}.csv", index=False)